In [1]:
from nn_compression.cv import CvModel
from nn_compression.nlp import LanguageModel
from nn_compression.networks import recursively_find_named_children
from nn_compression.quantisation import quantisable
from nn_compression.cv import unfold_depthwise_convolutions
import torch.nn as nn

In [2]:
net_enum = CvModel.MOBILENETV3_SMALL

nparams_dict = {}

def calc_overhead(net_enum, half_precision: bool = True):
    net = net_enum.load()
    vfn = net_enum.filter_fn()
    if half_precision:
        scale = 16
    else:
        scale = 32

    nquantisable = 0
    nparams = 0
    for n, l in recursively_find_named_children(net):
        if vfn(n) and quantisable(l):
            if isinstance(l, nn.Conv2d):
                nquantisable += 1  * l.groups
            else:
                nquantisable += 1
            nparams += l.weight.data.numel()

    nparams_dict[net_enum.value] = nparams
    return (nquantisable * scale) / nparams

In [3]:
d = {}
for n in CvModel:
    try:
        d[n.value] = calc_overhead(n)
    except: 
        continue
        d[n.value] = calc_overhead(n)
d[LanguageModel.PYTHIA_70M.value] = calc_overhead(LanguageModel.PYTHIA_70M)


/opt/homebrew/Caskroom/miniconda/base/envs/nn-compression/lib/python3.12/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/nn-compression/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
d

{'resnet18_cifar10': 2.8675808514421065e-05,
 'resnet34_cifar10': 2.7093173423403083e-05,
 'resnet50_cifar10': 3.616631592164056e-05,
 'resnet18_imagenet': 2.8656086839405558e-05,
 'resnet34_imagenet': 2.7083389757061994e-05,
 'resnet50_imagenet': 3.6154473740937504e-05,
 'resnet101_imagenet': 3.92500818968055e-05,
 'resnet152_imagenet': 4.2764236076240634e-05,
 'mobilenetv3_large': 0.0148662550685418,
 'mobilenetv3_small': 0.01583003145102287,
 'vgg16': 1.8504580114885686e-06,
 'convnext_tiny': 0.003738606086102931,
 'pythia-70m': 2.0345052083333332e-05}